In [4]:
!pip install transformers datasets sentencepiece


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 485.4/485.4 kB 8.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 11.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 12.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 9.3 MB/s eta 0:00:00


In [9]:
import os
from transformers import T5ForConditionalGeneration, T5Tokenizer
import ipywidgets as widgets
from google.colab import files


In [11]:
model_name = "t5-small"  # You can change this to 't5-base' or 't5-large' for better accuracy
tokenizer = T5Tokenizer.from_pretrained(model_name)
model = T5ForConditionalGeneration.from_pretrained(model_name)


In [10]:
def summarize_text(text, max_length=150, min_length=50):
    input_text = "summarize: " + text
    input_ids = tokenizer.encode(input_text, return_tensors="pt", max_length=512, truncation=True)

    summary_ids = model.generate(input_ids, max_length=max_length, min_length=min_length, length_penalty=2.0, num_beams=4, early_stopping=True)

    return tokenizer.decode(summary_ids[0], skip_special_tokens=True)


In [15]:
input_choice = widgets.Dropdown(
    options=["Enter Text", "Upload File"],
    description="Input Type:",
    layout=widgets.Layout(width="50%")
)

text_input = widgets.Textarea(
    placeholder="Enter text here...",
    layout=widgets.Layout(width="100%", height="150px")
)

upload_button = widgets.FileUpload(accept='.txt', multiple=False)

button = widgets.Button(description="Summarize")
output = widgets.Output()
file_output = widgets.Output()

def summarize_from_text():
    text = text_input.value.strip()
    if text:
        summary = summarize_text(text)
        return summary
    return "Please enter some text!"

def summarize_from_file(change):
    with file_output:
        file_output.clear_output()
        uploaded_file = next(iter(upload_button.value))
        content = upload_button.value[uploaded_file]["content"].decode("utf-8")
        summary = summarize_text(content)
        print("\nGenerated Summary:\n", summary)

upload_button.observe(summarize_from_file, names='_counter')

def on_button_click(b):
    with output:
        output.clear_output()
        if input_choice.value == "Enter Text":
            summary = summarize_from_text()
            print("\nGenerated Summary:\n", summary)
        else:
            print("Please upload a file.")

button.on_click(on_button_click)

def on_input_choice_change(change):
    output.clear_output()
    file_output.clear_output()
    display_widgets()

def display_widgets():
    display(input_choice)
    if input_choice.value == "Enter Text":
        display(text_input, button, output)
    else:
        display(upload_button, file_output)

input_choice.observe(on_input_choice_change, names="value")

display_widgets()


Dropdown(description='Input Type:', layout=Layout(width='50%'), options=('Enter Text', 'Upload File'), value='…

Textarea(value='', layout=Layout(height='150px', width='100%'), placeholder='Enter text here...')

Button(description='Summarize', style=ButtonStyle())

Output()